In [1]:
import os
import numpy as np
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
        gpus[int(0)],
        [tf.config.LogicalDeviceConfiguration(memory_limit=15000)])
        tf.config.experimental.set_visible_devices(gpus[int(0)], 'GPU')
        # tf.config.experimental.set_memory_growth(gpus[int(SELECTED_GPU)], True)
        # tf.config.experimental.se.set_per_process_memory_fraction(0.92)
        print(gpus[int(0)])
    except RuntimeError as e:
        print(e)
else:
    print('GPU not found!')

# import matplotlib.pyplot as plt

from scipy.stats import pearsonr, spearmanr

import datasets
from tqdm.auto import tqdm

from transformers import (
    AutoTokenizer,
    TFBertForSequenceClassification
)

2022-11-29 10:49:38.120146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 10:49:38.253376: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-29 10:49:38.855633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/modaresi/.conda/envs/globenc-venv/lib/
2022-11-29 10:49:38.855716: W tensorflow/

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2022-11-29 10:49:39.613647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 10:49:39.620550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 10:49:39.620701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
models_dir = "/home/modaresi/projects/globenc_analysis/outputs/models"
outputs_dir = "/home/modaresi/projects/globenc_analysis/outputs/HTAs"
DATA_SECTION = "validation"
configs = {
    "mnli": {
        "model_path": lambda epoch: f"{models_dir}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-{12272*epoch}/",
        "output_file_path": lambda epoch: f"{outputs_dir}/mnli_{DATA_SECTION}_bert-base-uncased_0001_SEED0042_checkpoint-{12272*epoch}.pkl",
        "hf_ds": "mnli",
    },
    "sst2": {
        "model_path": lambda epoch: f"{models_dir}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-{epoch*2105}/",
        "output_file_path": lambda epoch: f"{outputs_dir}/sst2_{DATA_SECTION}_bert-base-uncased_0001_SEED0042_checkpoint-{epoch*2105}.pkl",
        "hf_ds": "sst2",
    }
}

TASK = "sst2"
CONFIG = configs[TASK]
MAX_LENGTH = 128
SEED = 42
EPOCH = 5

In [4]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

MAX_LENGTH = 64

actual_task = "mnli" if TASK == "mnli-mm" else TASK
dataset = datasets.load_dataset("glue", actual_task)
# metric = datasets.load_metric('glue', actual_task)
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
SENTENCE1_KEY, SENTENCE2_KEY = task_to_keys[TASK]
tokenizer = None
sel_dataset = None
dataset

Reusing dataset glue (/opt/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [5]:
import pickle
import pathlib
def preprocess_function_wrapped(tokenizer):
    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[SENTENCE1_KEY],) if SENTENCE2_KEY is None else (examples[SENTENCE1_KEY], examples[SENTENCE2_KEY])
        )
        result = tokenizer(*args, padding=False, max_length=MAX_LENGTH, truncation=True)
        return result
    return preprocess_function

def token_id_to_tokens_mapper(tokenizer, sample):
    length = len(sample["input_ids"])
    return tokenizer.convert_ids_to_tokens(sample["input_ids"])[:length], length

def save_pickle(obj, path):
    pathlib.Path(os.path.dirname(path)).mkdir(parents=True, exist_ok=True) 
    with open(path, 'wb') as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Saved {path}")

In [6]:
path = CONFIG["model_path"](EPOCH)
model = TFBertForSequenceClassification.from_pretrained(path, from_pt=True, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(path, use_fast=True, max_length=128) if tokenizer is None else tokenizer
sel_dataset = dataset[DATA_SECTION].map(preprocess_function_wrapped(tokenizer), batched=True, batch_size=1024) if sel_dataset is None else sel_dataset
dataset_size = len(sel_dataset)

2022-11-29 10:50:03.302363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 10:50:03.308628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 10:50:03.309094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-29 10:50:03.309432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
HTAs = []
HTAs_angular = []

# @tf.function(input_signature=[{'input_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
#             'token_type_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
#             'attention_mask': tf.TensorSpec(shape=[1, None], dtype=tf.int32)}, 
#             tf.TensorSpec(shape=(1,), dtype=tf.int32)])
@tf.function(input_signature=[{'input_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
            'token_type_ids': tf.TensorSpec(shape=[1, None], dtype=tf.int32),
            'attention_mask': tf.TensorSpec(shape=[1, None], dtype=tf.int32)}])
def get_grads(inputs):
    with tf.GradientTape() as tape:
        outputs = model(inputs)
        input_embeds = outputs.hidden_states[0]
        # out_embeds = tf.gather(outputs.hidden_states[-1], i, batch_dims=1)
        out_embeds = outputs.hidden_states[-1][:, 0]

    grads = tape.batch_jacobian(out_embeds, input_embeds)
    inputXgradients = grads * tf.expand_dims(input_embeds, axis=1)
    return tf.linalg.norm(inputXgradients, axis=(1,3)), tf.reduce_sum(inputXgradients, axis=(1,3))

for idx in tqdm(range(dataset_size)):
    length = len(sel_dataset[idx]['input_ids'])
    hta = np.zeros((length), dtype=np.float32)
    hta_angular = np.zeros((length), dtype=np.float32)
    feats = {
            'input_ids': tf.constant([sel_dataset[idx]['input_ids']], dtype=tf.int32),
            'attention_mask': tf.constant([sel_dataset[idx]['attention_mask']], dtype=tf.int32),
            'token_type_ids': tf.constant([sel_dataset[idx]['token_type_ids']], dtype=tf.int32),
    }
    i = 0
    norms, angleDot = get_grads(feats) 
    # hta[i, :length] = np.linalg.norm(grads, axis=(1,3))
    # hta_angular[i, :length] = np.sum(grads, axis=(1,3))
    HTAs.append(norms.numpy())
    HTAs_angular.append(angleDot.numpy())

save_pickle({"HTAs": HTAs, "HTAs_angular": HTAs_angular}, CONFIG["output_file_path"](EPOCH))

  0%|          | 0/872 [00:00<?, ?it/s]

Saved /home/modaresi/projects/globenc_analysis/outputs/HTAs/sst2_validation_bert-base-uncased_0001_SEED0042_checkpoint-10525.pkl
